# Bisection Method

In [1]:
%reset -f
import numpy as np
import typing

def bisection(
        func: typing.Callable[[float], float], 
        boundary: list, 
        maxiter: int = 500, 
        tol: float = 1e-5
        ) -> float:
    """
    Returns root of given function in given boundary found by bisection method.
    Function should be scalar-valued and single-variable.
    """
    a = boundary[0] # lower bound
    b = boundary[1] # upper bound
    
    assert func(a)*func(b) < 0, "No root in given range."

    if func(a)*func(b) == 0:
        if func(a) == 0:
            return a
        else:
            return b
    else:
        for _ in range(maxiter):
            m = (a + b)/2 # midpoint

            if abs(func(m)) < tol:
                return m
            
            if func(a)*func(m) > 0:
                a = m
            else: # func(b)*func(m) > 0
                b = m
        
        return m

In [2]:
# Example of bisection method
import numpy as np

func = lambda x: np.sin(x) - np.log(x)
root = bisection(func, [1, 10])
print(root)

2.219104766845703


# Newton-Raphson Method

In [3]:
%reset -f
import numpy as np
import typing

from differentiation import diff

def newton_raphson(
        func: typing.Callable[[float], float],
        diff_func: typing.Callable[[float], float] = None,
        x0: float = 0.1,
        tol: float = 1e-8,
        maxiter: int = 500,
    ) -> float:
    """
    Returns root of given function found by Newton-Raphson method.
    Function should be scalar-valued and single-variable.
    """
    if diff_func is None: # Numerically differentiate
        x = x0
        for _ in range(maxiter):
            if abs(func(x)) < tol:
                return x
            x -= func(x)/diff(func, x)

        return x
    else:
        x = x0
        for _ in range(maxiter):
            if abs(func(x)) < tol:
                return x
            x -= func(x)/diff_func(x)
        
        return x

In [4]:
# Example of Newton-Raphson method
import numpy as np

func = lambda x: np.sin(x) - np.log(x)
diff_func = lambda x: np.cos(x) - 1 / x
root = newton_raphson(func)
print(root)

2.2191071489140937


<span style="color:orange;font-size:16pt">
Newton-Raphson Method for system of eqns (code below) failed to converge.
</span>

In [29]:
%reset -f
import numpy as np
import typing
from numpy import typing as nptyping

from differentiation import jacobi

def newton_raphson_multivar(
    func_vect: typing.List[typing.Callable[[nptyping.ArrayLike], nptyping.ArrayLike]],
    x0_vect: nptyping.ArrayLike,
    jacobian: typing.Callable[[nptyping.ArrayLike], np.ndarray] = None,
    tol: float = 1e-8,
    maxiter: int = 500,
) -> np.ndarray:
    if jacobian is None:
        x_vect = x0_vect
        for _ in range(maxiter):
            if np.linalg.norm(func_vect(x_vect)) < tol:
                return x_vect
            jacobian = jacobi(func_vect, x_vect)
            # Can use any system of equations solving method here
            delta_x_vect = -np.dot(jacobian, func_vect(x_vect))
            
            x_vect += delta_x_vect
        print("Warning: Newton-Raphson method did not converge within the specified number of iterations.")
        return x_vect
    else:
        x_vect = x0_vect

        for _ in range(maxiter):
            if np.linalg.norm(func_vect(x_vect)) < tol:
                return x_vect
            # Can use any system of equations solving method here
            delta_x_vect = np.matmul(np.linalg.inv(jacobian(x_vect)), -func_vect(x_vect))
            
            x_vect += delta_x_vect
        return x_vect


In [31]:
# Example of Newton-Raphson method for system of equations
func_vect = lambda x_vect: np.array(
    [x_vect[0] * np.cos(x_vect[1]) - 4, x_vect[0] * x_vect[1] - x_vect[0] - 5]
)
root = newton_raphson_multivar(func_vect, [0.1, 0.1])
print(root)

[-2.58570458e+11  3.65913580e+11]
